In [ ]:
%pip install rioxarray
%pip install xarray
%pip install earthengine-api rasterio
%pip install geopandas
%pip install gdal==$(gdal-config --version) geemap
%pip install setuptools

In [ ]:
import ee
ee.Authenticate()
ee.Initialize(project='btech-project-443813')

### Download NDVI Monthly

In [ ]:
import ee
import geemap
import os
from datetime import datetime, timedelta

ee.Initialize()

bounds = {
    'min_lon': 76.5,  # Western boundary of Delhi
    'max_lon': 77.5,  # Eastern boundary of Delhi
    'min_lat': 28.3,  # Southern boundary of Delhi
    'max_lat': 28.9   # Northern boundary of Delhi
}

start_date = datetime(2000, 1, 1)
end_date = datetime(2023, 12, 31)

output_dir = r"/home/stormej/dev/rainscale/data/ndvi/ndvi_monthly_temp"
os.makedirs(output_dir, exist_ok=True)

def get_monthly_ndvi(year,month):
    start = datetime(year, month, 1)
    end = (start + timedelta(days=32)).replace(day=1)
    
    ds = ee.ImageCollection('MODIS/006/MOD09GA').filterDate(start.strftime('%Y-%m-%d'), end.strftime('%Y-%m-%d'))
    
    def add_ndvi(image):
        ndvi = image.normalizedDifference(['sur_refl_b02', 'sur_refl_b01']).rename('NDVI')
        return image.addBands(ndvi)
    
    # Create a geometry from bounds
    geometry = ee.Geometry.Rectangle([bounds['min_lon'], bounds['min_lat'], bounds['max_lon'], bounds['max_lat']])
    
    monthly_ndvi = ds.map(add_ndvi).select('NDVI').mean().clip(geometry)
    return monthly_ndvi

def download_monthly_ndvi(year,month):
    date_str = datetime(year, month, 1).strftime('%Y-%m')
    ndvi = get_monthly_ndvi(year, month)
    filename = os.path.join(output_dir, f"ndvi_{date_str}.tif")
    geometry = ee.Geometry.Rectangle([bounds['min_lon'], bounds['min_lat'], bounds['max_lon'], bounds['max_lat']])
    
    geemap.ee_export_image(
        ndvi,
        filename=filename,
        scale=500,
        region=geometry.getInfo()['coordinates'],
        crs='EPSG:4326',
    )
    
    print(f"Downloaded NDVI for {date_str}")
    
def add_month(date_time):
    if date_time.month == 12:
        return date_time.replace(year=date_time.year + 1, month=1)
    else:
        return date_time.replace(month=date_time.month + 1)

current_date = start_date
while current_date <= end_date:
    download_monthly_ndvi(current_date.year, current_date.month)
    current_date = add_month(current_date)

print("All NDVI images downloaded successfully.")

### Resample NDVI at 0.25 degree

In [3]:
import os
import rasterio
from rasterio.enums import Resampling
from affine import Affine

input_dir = r"/home/stormej/dev/rainscale/data/ndvi/ndvi_monthly"
output_dir = r"/home/stormej/dev/rainscale/data/ndvi/ndvi_monthly_resampled_0.25"
os.makedirs(output_dir, exist_ok=True)

crs = 'EPSG:4326'
shape = (3, 5)
transform = Affine(0.25, 0, 76.5, 0, -0.25, 28.9)

for file in os.listdir(input_dir):
    if file.endswith('.tif'):
        path = os.path.join(input_dir, file)
        with rasterio.open(path) as src:
            resample = src.read(
                out_shape = (1, shape[0], shape[1]),
                resampling= Resampling.average,
            )
        parts = file.split('_')
        if len(parts) == 2:
            month = parts[1].replace('.tif', '')
        elif len(parts) == 3:
            month = parts[1]
        else:
            month = "unknown"
        
        output_filename = f"ndvi_resampled_{month}.tif"
        output_path = os.path.join(output_dir, output_filename)
        
        with rasterio.open(
            output_path,
            'w',
            driver='GTiff',
            height=shape[0],
            width=shape[1],
            count=1,
            dtype=resample.dtype,
            crs=crs,
            transform=transform,
        ) as dst:
            dst.write(resample[0], 1)
        
print("All NDVI images resampled successfully.")

All NDVI images resampled successfully.


### Resample NDVI to 0.01 degree

In [ ]:
import os
import rasterio
from rasterio.enums import Resampling
from affine import Affine

input_dir = r"/home/stormej/dev/rainscale/data/ndvi/ndvi_monthly"
output_dir = r"/home/stormej/dev/rainscale/data/ndvi/ndvi_monthly_resampled_0.01"
os.makedirs(output_dir, exist_ok=True)

crs = 'EPSG:4326'
width = int((bounds['max_lon'] - bounds['min_lon']) / 0.01)  # 100 pixels
height = int((bounds['max_lat'] - bounds['min_lat']) / 0.01)  # 60 pixels
shape = (height, width)
transform = Affine(0.01, 0, bounds['min_lon'], 0, -0.01, bounds['max_lat'])

for file in os.listdir(input_dir):
    if file.endswith('.tif'):
        path = os.path.join(input_dir, file)
        with rasterio.open(path) as src:
            resample = src.read(
                out_shape = (1, shape[0], shape[1]),
                resampling= Resampling.average,
            )
        parts = file.split('_')
        if len(parts) == 2:
            month = parts[1].replace('.tif', '')
        elif len(parts) == 3:
            month = parts[1]
        else:
            month = "unknown"
        
        output_filename = f"ndvi_resampled_{month}.tif"
        output_path = os.path.join(output_dir, output_filename)
        
        with rasterio.open(
            output_path,
            'w',
            driver='GTiff',
            height=shape[0],
            width=shape[1],
            count=1,
            dtype=resample.dtype,
            crs=crs,
            transform=transform,
        ) as dst:
            dst.write(resample[0], 1)
        
print("All NDVI images resampled successfully.")

All NDVI images resampled successfully.


### Resample NDVI to 0.1 degree i.e 10 km

In [3]:
import os
import rasterio
from rasterio.enums import Resampling
from affine import Affine
import ee

ee.Initialize()

bounds = {
    'min_lon': 76.5,  # Western boundary of Delhi
    'max_lon': 77.5,  # Eastern boundary of Delhi
    'min_lat': 28.3,  # Southern boundary of Delhi
    'max_lat': 28.9   # Northern boundary of Delhi
}

input_dir = r"/home/stormej/dev/rainscale/data/ndvi/ndvi_monthly"
output_dir = r"/home/stormej/dev/rainscale/data/ndvi/ndvi_monthly_resampled_0.1"
os.makedirs(output_dir, exist_ok=True)

crs = 'EPSG:4326'
width = int((bounds['max_lon'] - bounds['min_lon']) / 0.1)  # 10 pixels
height = int((bounds['max_lat'] - bounds['min_lat']) / 0.1)  # 6 pixels
shape = (height, width)
transform = Affine(0.1, 0, bounds['min_lon'], 0, -0.1, bounds['max_lat'])

for file in os.listdir(input_dir):
    if file.endswith('.tif'):
        path = os.path.join(input_dir, file)
        with rasterio.open(path) as src:
            resample = src.read(
                out_shape = (1, shape[0], shape[1]),
                resampling= Resampling.average,
            )
        parts = file.split('_')
        if len(parts) == 2:
            month = parts[1].replace('.tif', '')
        elif len(parts) == 3:
            month = parts[1]
        else:
            month = "unknown"
        
        output_filename = f"ndvi_resampled_{month}.tif"
        output_path = os.path.join(output_dir, output_filename)
        
        with rasterio.open(
            output_path,
            'w',
            driver='GTiff',
            height=shape[0],
            width=shape[1],
            count=1,
            dtype=resample.dtype,
            crs=crs,
            transform=transform,
        ) as dst:
            dst.write(resample[0], 1)
        
print("All NDVI images resampled successfully.")

All NDVI images resampled successfully.
